In [1]:
import os
import json
import pandas as pd
import nltk
pd.options.mode.chained_assignment = None
%config Completer.use_jedi = False
import re
import fasttext
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
d2vmodel = fasttext.load_model('cc.en.300.bin')

import spacy
import spacy_stanza
from spacy_fastlang import LanguageDetector

def _proc_text(text, nlp):
    text_lang, text_clnd, text_norm, text_lemm = 'xx', '', '', ''

    if text is None:
        return [text_lang, text_clnd, text_norm, text_lemm]

    text = str(text)
    #text = text.lower()
    text = re.sub(r'(\d+)', r' \1 ', text)
    text = text+'.'
    text = re.sub('[\!\?\n\.]+', '. ', text)
    text = re.sub('[^a-zA-Z0-9\'\.]', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = re.sub('(\s\.)', '.', text)
    text = text.strip()

    doc = nlp(text)

    sentences_clnd, sentences_norm, sentences_lemm = [], [], []
    for sentence in doc.sents:
        cleaned, normalized, lemmatized = [], [], []
        for word in sentence:
            print(word, word.norm_, word.lemma_)
            word_norm = word.norm_.lower()
            word_lemm = word.lemma_.lower()
            word_clean = re.sub('[^a-zA-Z0-9]', '', word_norm).strip()
            if word_norm == '\'s' and word_lemm == 'be':
                cleaned.append('is')
            elif word_norm == 'not' and word_lemm == 'n\'t':
                cleaned.append('not')
                normalized.append('not')
                lemmatized.append('not')
            elif len(word_clean) < 2:
                continue
            elif (word.is_stop is False or word_clean in ['no', 'not'] or word.dep_ == 'neg'):
                cleaned.append(word_clean)
                normalized.append(word_norm)
                lemmatized.append(word_lemm)
            else:
                cleaned.append(word_clean)

        sentence_clnd = ' '.join(cleaned)
        sentence_clnd = re.sub('\s+', ' ', sentence_clnd).strip()
        if len(sentence_clnd) > 0:
            sentences_clnd.append(sentence_clnd)

        sentence_norm = ' '.join(normalized)
        sentence_norm = re.sub('\s+', ' ', sentence_norm).strip()
        if len(sentence_norm) > 0:
            sentences_norm.append(sentence_norm)

        sentence_lemm = ' '.join(lemmatized)
        sentence_lemm = re.sub('\s+', ' ', sentence_lemm).strip()
        if len(sentence_lemm) > 0:
            sentences_lemm.append(sentence_lemm)

    if len(sentences_clnd):
        text_clnd = ' . '.join(sentences_clnd)+' .'
        text_lang = doc._.language
    if len(sentences_norm):
        text_norm = ' . '.join(sentences_norm)+' .'
    if len(sentences_lemm):
        text_lemm = ' . '.join(sentences_lemm)+' .'

    return [text_lang, text_clnd, text_norm, text_lemm]

[nltk_data] Downloading package punkt to /home/kr/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/kr/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
nlp_cln = spacy.load('en_core_web_md', disable=['ner', 'parser', 'spacytextblob'])
nlp_cln.add_pipe('language_detector', first=True)
nlp_cln.add_pipe('sentencizer', last=True)

print(nlp_cln.pipe_names)

#text = "apps tickets"

#print(_proc_text(text, nlp_cln))

['language_detector', 'tok2vec', 'tagger', 'attribute_ruler', 'lemmatizer', 'sentencizer']


In [3]:
import spacy #load spacy
from spacy import displacy
from spacytextblob.spacytextblob import SpacyTextBlob
from nltk import Tree

nlp = spacy.load("en_core_web_lg")
nlp.add_pipe('spacytextblob')

def opinion_mining_method1(sentence):
    '''
    input: dictionary and sentence
    function: appends dictionary with new features if the feature
              did not exist previously,then updates sentiment to
              each of the new or existing features
    output: updated dictionary
    '''
    sentence = nlp(sentence)
    #displacy.render(sentence, style='dep')
    
    def tok_format(tok):
        return "_".join([tok.orth_, tok.pos_])


    def to_nltk_tree(node):
        if node.n_lefts + node.n_rights > 0:
            return Tree(tok_format(node), [to_nltk_tree(child) for child in node.children])
        else:
            return tok_format(node)
    
    [to_nltk_tree(sent.root).pretty_print() for sent in sentence.sents]

    opinion_words = dict()

    for assessment in sentence._.assessments:
        sentiment_info = dict()
        sentiment_info['keyword'] = assessment[0][-1]
        sentiment_info['assessment'] = ' '.join(assessment[0])
        sentiment_info['score'] = assessment[1]
        opinion_words[assessment[0][-1]] = sentiment_info
                
    def get_target_tokens(token):
        target_tokens = []
        for child in token.children:
            #if child.dep_ in ['acl', 'relcl', 'compound', 'attr', 'xcomp']:
            if child.dep_ not in ['advmod', 'amod'] and child.pos_ not in ['ADP', 'AUX', 'CONJ', 'DET', 'NUM', 'PART', 'PRON', 'PUNCT', 'SCONJ', 'SYM', 'X']:
                target_tokens.append(child.text)

        target_tokens.append(token.text)

        #if token.head.dep_ in ['acl', 'relcl', 'compound', 'attr', 'xcomp', 'dobj']:
        if token.head.dep_ not in ['advmod', 'amod'] and token.head.pos_ not in ['ADP', 'AUX', 'CONJ', 'DET', 'NUM', 'PART', 'PRON', 'PUNCT', 'SCONJ', 'SYM', 'X']:
            target_tokens.append(token.head.text)
            for child in token.head.children:
                #if child.dep_ in ['acl', 'relcl', 'compound', 'attr']:
                if child.dep_ not in ['advmod', 'amod'] and child.pos_ not in ['ADP', 'AUX', 'CONJ', 'DET', 'NUM', 'PART', 'PRON', 'PUNCT', 'SCONJ', 'SYM', 'X'] and child.text not in target_tokens:
                    target_tokens.append(child.text)

        return target_tokens
    
    print(opinion_words)    
    
    all_sentiments = dict()
    for token in sentence:
        #print(token.text, token.head.text, token.dep_, token.pos_)
        # check if the word is an opinion word, then assign sentiment
        if token.text in opinion_words:
            target_tokens = []
            assessment_tokens = []
            sentiment = 0 #if token.text in pos else -1
            # if target is an adverb modifier (i.e. pretty, highly, etc.)
            # but happens to be an opinion word, ignore and pass
            if (token.dep_ == "advmod"):
                continue
            elif (token.dep_ == "amod"):
                target_tokens = get_target_tokens(token.head)
                sent_dict = dict()
                sent_dict['target'] = ' '.join(target_tokens)
                sent_dict['assessment'] = opinion_words[token.text]['assessment'].strip()
                sent_dict['sentiment'] = opinion_words[token.text]['score']
                sent_dict['debug'] = 0
                all_sentiments[token.head.text] = sent_dict
            # for opinion words that are adjectives, adverbs, verbs...
            else:

                for child in token.children:
                    # if there's a adj modifier (i.e. very, pretty, etc.) add more weight to sentiment
                    # This could be better updated for modifiers that either positively or negatively emphasize
                    if ((child.dep_ == "amod") or (child.dep_ == "advmod")) and (child.text in opinion_words):
                        sentiment *= 1.5
                    # check for negation words and flip the sign of sentiment
                    if child.dep_ == "neg":
                        sentiment *= -1
                for child in token.children:
                    # if verb, check if there's a direct object
                    if (token.pos_ == "VERB") & (child.dep_ == "dobj"):                        
                        sent_dict = dict()
                        sent_dict['target'] = child.text
                        sent_dict['assessment'] = opinion_words[token.text]['assessment'].strip()
                        sent_dict['sentiment'] = opinion_words[token.text]['score']
                        sent_dict['debug'] = 1
                        all_sentiments[child.text] = sent_dict
                        #sent_dict[child.text] = sentiment
                        # check for conjugates (a AND b), then add both to dictionary
                        subchildren = []
                        conj = 0
                        for subchild in child.children:
                            if subchild.text == "and":
                                conj=1
                            if (conj == 1) and (subchild.text != "and"):
                                subchildren.append(subchild.text)
                                conj = 0
                        for subchild in subchildren:
                            sent_dict = dict()
                            sent_dict['target'] = subchild
                            sent_dict['assessment'] = opinion_words[token.text]['assessment'].strip()
                            sent_dict['sentiment'] = opinion_words[token.text]['score']
                            sent_dict['debug'] = 2
                            all_sentiments[subchild] = sent_dict

                # check for negation
                for child in token.head.children:
                    if ((child.dep_ == "amod") or (child.dep_ == "advmod")) and (child.text in opinion_words):
                        sentiment *= 1.5
                    # check for negation words and flip the sign of sentiment
                    if (child.dep_ == "neg"): 
                        sentiment *= -1
                
                # check for nouns
                for child in token.head.children:
                    noun = ""
                    if (child.pos_ == "NOUN") and (child.text not in all_sentiments):
                        noun = child.text
                        # Check for compound nouns
                        for subchild in child.children:
                            if subchild.dep_ == "compound":
                                noun = subchild.text + " " + noun
                        sent_dict = dict()
                        sent_dict['target'] = noun
                        sent_dict['assessment'] = opinion_words[token.text]['assessment'].strip()
                        sent_dict['sentiment'] = opinion_words[token.text]['score']
                        sent_dict['debug'] = 3
                        all_sentiments[noun] = sent_dict
                        
                if not all_sentiments:
                    sent_dict = dict()
                    sent_dict['target'] = 'app'
                    sent_dict['assessment'] = opinion_words[token.text]['assessment'].strip()
                    sent_dict['sentiment'] = opinion_words[token.text]['score']
                    sent_dict['debug'] = 4
                    all_sentiments[noun] = sent_dict
    
    return all_sentiments

In [4]:
text = "Nice ride but glitchy app"
#text = "Super expensive ..they looted me for 500 rupees for just 10 minutes ride.."
#text = "Yulu had not gave my refund money which paid me though payu but money didn't receive to bank a/c if I didn't receive my yulu refund I will file an FIR"
#text = they are looting miney in the super saver packs. No charged vehicle are available near peak point. Bad service high prices. Complained but only excuses. Why to charge per minute in banglore traffic, better to walk or owe cycle instead of spending it on yulu. High price low service."
#text = "Great stupid developer making this app, Developer making this app only for owner, nothing any solution for customer."
#text = "Best battery bike ever worth off money \ud83d\udcb8 \ud83d\udcb8\ud83d\udcb8 I had used in banglore... For one day it's very useful... Am in chennai.... But yulu is not there in chennai kindly make tie up with this government it's very useful for us........"
#text = "Best for daily commuters in Bengaluru"
#text = "Nice experience. Arrangement is clear and smooth. Cycle quality is good. Only suggestion to indicate battery life once ride starts."
#text = "Great ride i reached office on time today exactly at 10am... Fabulous i loved the ride... Charges should be less... Its not distinguished how the fare is calculated..."
#text = "Good for nearby places.....but your charges is high for an electric vehicle. It is like 1rs per min....pls consider"
#text = "App is crashing like anything even after updating it.."
#text = "Waste of money 3rd class app very bad service"
opinion_mining_method1(text)

         ride_NOUN            
    _________|__________       
   |         |       app_NOUN 
   |         |          |      
Nice_ADJ but_CCONJ glitchy_ADJ

{'nice': {'keyword': 'nice', 'assessment': 'nice', 'score': 0.6}}


{}

In [5]:
import pandas as pd
import numpy as np
import nltk
from nltk import Tree
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
import spacy
from spacy import displacy
from spacytextblob.spacytextblob import SpacyTextBlob
import stanza
#import spacy-stanza

def get_dep_node_stanza(finaltxt):
    nlp = stanza.Pipeline(lang='en', processors={'tokenize': 'spacy'})
    doc = nlp(finaltxt)  # Object of Stanza NLP Pipeleine

    # Getting the dependency relations betwwen the words
    dep_node = []
    for dep_edge in doc.sentences[0].dependencies:
        dep_node.append([dep_edge[2].text, dep_edge[0].id, dep_edge[1]])

    return dep_node

def get_dep_node_spacy(finaltxt):
    nlp = spacy.load("en_core_web_lg")
    #nlp.add_pipe('spacytextblob')

    doc = nlp(finaltxt)  # Object of Stanza NLP Pipeleine

    # Getting the dependency relations betwwen the words
    dep_node = []
    for dep_edge in doc.sents[0].dependencies:
        dep_node.append([dep_edge[2].text, dep_edge[0].id, dep_edge[1]])

    return dep_node

def opinion_mining_method2(txt, stop_words, nlp='stanza'):
    catNN = ['NN', 'NNS', 'NNP', 'NNPS']
    
    txt = txt.lower() # LowerCasing the given Text
    sentList = nltk.sent_tokenize(txt) # Splitting the text into sentences

    fcluster = []
    totalfeatureList = []
    finalcluster = []
    dic = {}
    print(sentList)

    for line in sentList:
        newtaggedList = []
        txt_list = nltk.word_tokenize(line) # Splitting up into words
        taggedList = nltk.pos_tag(txt_list) # Doing Part-of-Speech Tagging to each word

        newwordList = []
        flag = 0
        for i in range(0,len(taggedList)-1):
            print(taggedList[i])
            if(taggedList[i][1] in catNN and taggedList[i+1][1] in catNN): # If two consecutive words are Nouns then they are joined together
                newwordList.append(taggedList[i][0]+'_'+taggedList[i+1][0])
                flag=1
            else:
                if(flag==1):
                    flag=0
                    continue
                newwordList.append(taggedList[i][0])
                if(i==len(taggedList)-2):
                    print(taggedList[i+1])
                    newwordList.append(taggedList[i+1][0])

        finaltxt = ' '.join(word for word in newwordList) 
        new_txt_list = nltk.word_tokenize(finaltxt)
        wordsList = [w for w in new_txt_list if not w in stop_words]
        taggedList = nltk.pos_tag(wordsList)
        
        dep_node = []
        if nlp == 'stanza':
            dep_node = get_dep_node_stanza(finaltxt)
        elif nlp == 'stanfordnlp':
            dep_node = get_dep_node_stanfordnlp(finaltxt)
        elif nlp == 'spacy':
            dep_node = get_dep_node_spacy(finaltxt)
            
        # Coverting it into appropriate format
        for i in range(0, len(dep_node)):
            if (int(dep_node[i][1]) != 0):
                dep_node[i][1] = newwordList[(int(dep_node[i][1]) - 1)]
        print(dep_node, '\n')
        
        featureList = []
        for i in taggedList:
            if(i[1]=='JJ' or i[1]=='JJR' or i[1]=='RB' or i[1] in catNN):
                featureList.append(list(i)) # For features for each sentence
                totalfeatureList.append(list(i)) # Stores the features of all the sentences in the text
        print(totalfeatureList)

        for i in featureList:
            filist = []
            for j in dep_node:
                if((j[0]==i[0] or j[1]==i[0]) and (j[2] in ["nsubj", "acl:relcl", "obj", "obl", "dobj", "agent", "advmod", "amod", "neg", "prep_of", "acomp", "xcomp", "compound"])):
                    if(j[0]==i[0]):
                        filist.append(j[1])
                    else:
                        filist.append(j[0])
            fcluster.append([i[0], filist])
        print(fcluster)
            
    for i in totalfeatureList:
        dic[i[0]] = i[1]
    
    for i in fcluster:
        if(dic[i[0]] in catNN):
            finalcluster.append(i)
        
    return finalcluster

In [6]:
stop_words = []#stopwords.words('english')

text = "Nice ride but glitchy app"
#text = "Super expensive ..they looted me for 500 rupees for just 10 minutes ride.."
#text = "Yulu had not gave my refund money which paid me though payu but money didn't receive to bank a/c if I didn't receive my yulu refund I will file an FIR"
#text = they are looting miney in the super saver packs. No charged vehicle are available near peak point. Bad service high prices. Complained but only excuses. Why to charge per minute in banglore traffic, better to walk or owe cycle instead of spending it on yulu. High price low service."
#text = "Great stupid developer making this app, Developer making this app only for owner, nothing any solution for customer."
#text = "Best battery bike ever worth off money \ud83d\udcb8 \ud83d\udcb8\ud83d\udcb8 I had used in banglore... For one day it's very useful... Am in chennai.... But yulu is not there in chennai kindly make tie up with this government it's very useful for us........"
#text = "Best for daily commuters in Bengaluru"
#text = "Nice experience. Arrangement is clear and smooth. Cycle quality is good. Only suggestion to indicate battery life once ride starts."
#text = "Great ride i reached office on time today exactly at 10am... Fabulous i loved the ride... Charges should be less... Its not distinguished how the fare is calculated..."
#text = "Good for nearby places.....but your charges is high for an electric vehicle. It is like 1rs per min....pls consider"
#text = "App is crashing like anything even after updating it.."
#text = "Waste of money 3rd class app very bad service"
opinion_mining_method2(text, stop_words, 'stanza')

2022-02-07 13:38:41 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | spacy     |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2022-02-07 13:38:41 INFO: Use device: gpu
2022-02-07 13:38:41 INFO: Loading: tokenize


['nice ride but glitchy app']
('nice', 'JJ')
('ride', 'NN')
('but', 'CC')
('glitchy', 'NN')


2022-02-07 13:38:41 INFO: Loading: pos
2022-02-07 13:38:45 INFO: Loading: lemma
2022-02-07 13:38:45 INFO: Loading: depparse
2022-02-07 13:38:45 INFO: Loading: sentiment
2022-02-07 13:38:45 INFO: Loading: constituency
2022-02-07 13:38:46 INFO: Loading: ner
2022-02-07 13:38:46 INFO: Done loading processors!


[['nice', 'ride', 'amod'], ['ride', 0, 'root'], ['but', 'glitchy_app', 'cc'], ['glitchy_app', 'ride', 'conj']] 

[['nice', 'JJ'], ['ride', 'NN'], ['glitchy_app', 'NN']]
[['nice', ['ride']], ['ride', ['nice']], ['glitchy_app', []]]


[['ride', ['nice']], ['glitchy_app', []]]

In [7]:
import stanza
import spacy
import spacy_stanza
from spacy import displacy
from spacytextblob.spacytextblob import SpacyTextBlob
import nltk
from nltk import Tree
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer 

def get_dependencies_spacy(sentence):
    nlp = spacy.load("en_core_web_lg")
    #nlp.add_pipe('spacytextblob')

    doc = nlp(sentence)  # Object of Stanza NLP Pipeleine

    dep_node = []
    tokens = doc.to_json()['tokens']
    for token in tokens:
        dep_node.append([sentence[token['start']:token['end']], token['id'], token['dep']])

    return dep_node

def get_tokens_spacy(sentence):
    #nlp = spacy_stanza.load_pipeline("en")
    nlp = spacy.load("en_core_web_lg")
    tokens = nlp(sentence)
    
    sentence_words, sentence_tags = [], []
    for token in tokens:
        if token.is_stop is False:
            sentence_tags.append((token.text, token.tag_))
        sentence_words.append(token.text)
        
    return sentence_words, sentence_tags

def get_tokens_nltk(sentence):
    stop_words = stopwords.words('english')

    tag_list = nltk.pos_tag(nltk.word_tokenize(sentence)) # Doing Part-of-Speech Tagging to each word

    sentence_words = []
    flag = 0
    for i in range(0,len(tag_list)-1):
        if(tag_list[i][1] in catNN and tag_list[i+1][1] in catNN): # If two consecutive words are Nouns then they are joined together
            sentence_words.append(tag_list[i][0]+'_'+tag_list[i+1][0])
            flag=1
        else:
            if(flag==1):
                flag=0
                continue
            sentence_words.append(tag_list[i][0])
            if(i==len(tag_list)-2):
                print(tag_list[i+1])
                sentence_words.append(tag_list[i+1][0])

    sentence_text = ' '.join(word for word in sentence_words) 
    word_list = nltk.word_tokenize(sentence_text)
    tag_list = [w for w in word_list if not w in stop_words]
    sentence_tags = nltk.pos_tag(tag_list)
    
    return sentence_words, sentence_tags

In [8]:
import stanza
import nltk
from nltk.corpus import stopwords
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

nlp = stanza.Pipeline(lang='en')
stop_words = stopwords.words('english')

sentiment_nlp = spacy.load('en_core_web_lg')
sentiment_nlp.add_pipe('spacytextblob')

2022-02-07 13:38:50 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2022-02-07 13:38:50 INFO: Use device: gpu
2022-02-07 13:38:50 INFO: Loading: tokenize
2022-02-07 13:38:50 INFO: Loading: pos
2022-02-07 13:38:50 INFO: Loading: lemma
2022-02-07 13:38:50 INFO: Loading: depparse
2022-02-07 13:38:50 INFO: Loading: sentiment
2022-02-07 13:38:50 INFO: Loading: constituency
2022-02-07 13:38:50 INFO: Loading: ner
2022-02-07 13:38:51 INFO: Done loading processors!


In [9]:
import re
import stanza
import nltk
from nltk.corpus import stopwords
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
import networkx as nx

nlp = stanza.Pipeline(lang='en')
stop_words = stopwords.words('english')

sentiment_nlp = spacy.load('en_core_web_lg')
sentiment_nlp.add_pipe('spacytextblob')

catNN = ['NN', 'NNS', 'NNP', 'NNPS']


def get_graph(doc):
    edges = []
    for token in doc:
        for child in token.children:
            edges.append(('{0}'.format(token.lower_), '{0}'.format(child.lower_)))
        #edges.append((token[0], token[1]))
    #print(edges)
    graph = nx.Graph(edges)
    return graph
    
def get_path(graph, aspect, qualifier):
    return nx.shortest_path_length(graph, source=aspect, target=qualifier), nx.shortest_path(graph, source=aspect, target=qualifier)


def process_stanza(sentence, nlp, stop_words, combine_nouns=False):
    doc = nlp(sentence)
    if combine_nouns:
        org_words = doc.sentences[0].words
        new_words = []
        j = 0
        for i in range(0, len(org_words)):
            if org_words[i].xpos in catNN:
                if j > 0:
                    j = j-1
                    continue
                word_NN = ''
                while(i+j < len(org_words) and org_words[i+j].xpos in catNN):
                    word_NN += org_words[i+j].text + '_'
                    j += 1
                new_words.append(word_NN[0:-1])
            else:
                new_words.append(org_words[i].text)
                j = 0
        sentence = ' '.join(new_words) 
        doc = nlp(sentence)
    
    words, targets, all_targets = [], {}, []
    for word in doc.sentences[0].words:
        #print(word.text, word.xpos)
        if word.text not in stop_words and word.xpos in catNN:#+['JJ', 'JR', 'RB']:
            targets[word.text] = word.xpos
            all_targets.append([word.text, word.xpos])
        words.append(word.text)
    #print(targets, all_targets)
        
    dependencies = []
    for dep_edge in doc.sentences[0].dependencies:
        if dep_edge[1] != 'root':
            dependencies.append([dep_edge[2].text, dep_edge[0].id, dep_edge[1]])

    # Coverting it into appropriate format
    for i in range(0, len(dependencies)):
        if (int(dependencies[i][1]) != 0):
            dependencies[i][1] = words[(int(dependencies[i][1]) - 1)]

    return words, targets, dependencies

def extract_aspects(sentence, nlp, stop_words, combine_nouns=False):
    words, targets, dependencies = process_stanza(sentence, nlp, stop_words, combine_nouns)
        
    sentence = ' '.join(words)
    sentiment = sentiment_nlp(sentence)
    sentiment_terms = {}
    for item in sentiment._.assessments:
        sentiment_terms[item[0][-1]] = [item[1], ' '.join(item[0])]
    
    sentence_graph = get_graph(sentiment)#dependencies)

    dependencies = [dep for dep in dependencies if dep[0] not in stop_words and dep[1] not in stop_words]    

    targets_assessments = {}
    for target in targets:
        if targets[target] not in catNN:#+['JJ','JR','RB']:
            continue
        target_text = target
        assessments = []
        for dependency in dependencies:
            #if dependency[2] in ['det', 'case', 'root', 'acl:relcl']:
            #if not dependency[2] in ["nsubj", "acl", "recl", "acl:relcl", "obj", "obl", "dobj", "agent", "advmod", "amod", "neg", "prep_of", "acomp", "xcomp", "compound"]:
            #if not dependency[2] in ["acl", "recl", "obj", "obl", "dobj", "agent", "advmod", "amod", "neg", "prep_of", "acomp", "xcomp", "compound"]:
            #if not dependency[2] in ["nsubj", "acl:relcl", "obj", "obl", "dobj", "agent", "advmod", "amod", "neg", "prep_of", "acomp", "xcomp", "compound"]:
                #continue
            #if dependency[0]==target and dependency[1] in targets:
                #target_text = target_text + ' ' + dependency[1]
            #elif dependency[1]==target and dependency[0] in targets:
                #target_text = target_text + ' ' + dependency[0]
            if dependency[0]==target and dependency[1] not in targets:# and dependency[1] not in stop_words:
                glength, gpath = get_path(sentence_graph, dependency[0], dependency[1])
                #print(glength, gpath)
                assessments.append(dependency[1])
            elif dependency[1]==target and dependency[0] not in targets:# and dependency[0] not in stop_words:
                glength, gpath = get_path(sentence_graph, dependency[1], dependency[0])
                #print(glength, gpath)
                assessments.append(dependency[0])
        if len(assessments) == 0:
            continue
        if targets[target] in catNN:#+['JJ','JR','RB']:
            target_text = " ".join([token.lemma_ for token in sentiment_nlp(target_text)])
            targets_assessments[target] = dict()
            targets_assessments[target]['target_text'] = target_text
            targets_assessments[target]['assessment_text'] = ' '.join(assessments)
            sent_terms = [term for term in assessments if term in sentiment_terms]
            if len(sent_terms) == 0:
                targets_assessments[target]['sentiment'] = 'unknown'
            elif sentiment_terms[sent_terms[0]][0] >= 0.33:
                targets_assessments[target]['sentiment'] = 'positive'
                #targets_assessments[target]['assessment_text'] = re.sub(sent_terms[0], sentiment_terms[sent_terms[0]][1], targets_assessments[target]['assessment_text'])
            elif sentiment_terms[sent_terms[0]][0] <= -0.33:
                targets_assessments[target]['sentiment'] = 'negative'
                #targets_assessments[target]['assessment_text'] = re.sub(sent_terms[0], sentiment_terms[sent_terms[0]][1], targets_assessments[target]['assessment_text'])
            else:
                targets_assessments[target]['sentiment'] = 'mixed'
                #targets_assessments[target]['assessment_text'] = re.sub(sent_terms[0], sentiment_terms[sent_terms[0]][1], targets_assessments[target]['assessment_text'])

    return targets_assessments

2022-02-07 13:38:55 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2022-02-07 13:38:55 INFO: Use device: gpu
2022-02-07 13:38:55 INFO: Loading: tokenize
2022-02-07 13:38:55 INFO: Loading: pos
2022-02-07 13:38:55 INFO: Loading: lemma
2022-02-07 13:38:55 INFO: Loading: depparse
2022-02-07 13:38:55 INFO: Loading: sentiment
2022-02-07 13:38:55 INFO: Loading: constituency
2022-02-07 13:38:55 INFO: Loading: ner
2022-02-07 13:38:56 INFO: Done loading processors!


In [10]:
#text = "nice yulu ride but glitchy app"
#text = "Super expensive ..they looted me for 500 rupees for just 10 minutes ride.."
#text = "yulu had not gave my refund money which paid me though payu but money didn't receive to bank a/c if I didn't receive my yulu refund I will file an fir"
#text = "they are looting miney in the super saver packs. No charged vehicle are available near peak point. Bad service high prices. Complained but only excuses. Why to charge per minute in banglore traffic, better to walk or owe cycle instead of spending it on yulu. High price low service."
#text = "great stupid developer making this app, geveloper making this app only for owner, nothing any solution for customer."
#text = "best battery bike ever worth off money \ud83d\udcb8 \ud83d\udcb8\ud83d\udcb8 I had used in banglore... For one day it's very useful... Am in chennai.... But yulu is not there in chennai kindly make tie up with this government it's very useful for us........"
#text = "best for daily commuters in bengaluru"
#text = "nice experience. arrangement is clear and smooth. Cycle quality is good. Only suggestion to indicate battery life once ride starts."
#text = "great ride i reached office on time today exactly at 10am... Fabulous i loved the ride... Charges should be less... Its not distinguished how the fare is calculated..."
#text = "good for nearby places.....but your charges is high for an electric vehicle. It is like 1rs per min....pls consider"
#text = "app is crashing like anything even after updating it.."
text = "waste of money 3rd class app very bad service"

result = extract_aspects(text, nlp, stop_words, combine_nouns=False)
#print(result)

result = extract_aspects(text, nlp, stop_words, combine_nouns=True)
#print(result)

import pandas as pd
values = pd.DataFrame(result.values())

values

,target_text,assessment_text,sentiment
0,class_app,3rd,mixed
1,service,bad,negative


In [11]:
import re
import spacy
import spacy_stanza
from spacytextblob.spacytextblob import SpacyTextBlob

nlp = spacy_stanza.load_pipeline("en")
nlp.add_pipe('spacytextblob')

catNN = ['NN', 'NNS', 'NNP', 'NNPS']#+['JJ', 'JR', 'RB']

def process_spacy(doc):
    words, targets, all_targets = [], {}, []
    for word in doc:
        if word.tag_ in catNN:
            targets[word.text] = [word.tag_, word.lemma_]
            all_targets.append([word.text, word.tag_, word.lemma_])
        words.append(word.text)
        
    edges = []
    for token in doc:
        token_info = [token.text, token.lemma_, (token.is_stop and token.text not in ['no', 'not']), token.pos_, token.tag_]
        token_info = [token.text, token.lemma_, False, token.pos_, token.tag_]
        for child in token.children:
            child_info = [child.text, child.lemma_, (child.is_stop and child.text not in ['no', 'not']), child.pos_, child.tag_]
            child_info = [child.text, child.lemma_, False, child.pos_, child.tag_]
            edges.append([child.dep_, token_info, child_info])

    return words, targets, edges

def extract_aspects(sentence, nlp):
    doc = nlp(sentence)
    words, targets, edges = process_spacy(doc)

    sentiment_terms = {}
    for item in doc._.assessments:
        sentiment_terms[item[0][-1]] = [item[1], ' '.join(item[0])]
    #print(sentiment_terms)
    targets_assessments = {}
    for target in targets:
        if targets[target][0] not in catNN:
            continue
        target_text = targets[target][1]
        assessments = []
        for edge in edges:
            if edge[0] in ['det', 'case', 'root', 'acl:relcl'] and 'no' not in edge[1]+edge[2]:
            #if not dependency[2] in ["nsubj", "acl", "recl", "acl:relcl", "obj", "obl", "dobj", "agent", "advmod", "amod", "neg", "prep_of", "acomp", "xcomp", "compound"]:
            #if not dependency[2] in ["acl", "recl", "obj", "obl", "dobj", "agent", "advmod", "amod", "neg", "prep_of", "acomp", "xcomp", "compound"]:
            #if not dependency[2] in ["nsubj", "acl:relcl", "obj", "obl", "dobj", "agent", "advmod", "amod", "neg", "prep_of", "acomp", "xcomp", "compound"]:
                continue
            if edge[1][0] == target and edge[2][2] is False:# and dependency[1] not in targets:#
                if edge[2][0] in targets:# and dependency[2] in ['conj', 'compound', 'nmod']:
                    target_text = target_text + ' ' + edge[2][1]
                else:
                    assessments.append(edge[2][0])
            elif edge[2][0] == target and edge[1][2] is False:# and dependency[0] not in targets:#
                if edge[1][0] in targets:# and dependency[2] in ['conj', 'compound', 'nmod']:
                    target_text = edge[1][1] + ' ' + target_text
                else:
                    assessments.append(edge[1][0])
            
        if len(assessments) == 0:
            continue

        targets_assessments[target] = dict()
        targets_assessments[target]['target_text'] = target_text
        targets_assessments[target]['assessment_text'] = assessments
        sent_terms = [term for term in assessments if term in sentiment_terms]
        if len(sent_terms) == 0:
            targets_assessments[target]['sentiment'] = 'unknown'
            targets_assessments[target]['sentiment_text'] = []
        elif sentiment_terms[sent_terms[0]][0] >= 0.33:
            targets_assessments[target]['sentiment'] = 'positive'
            targets_assessments[target]['sentiment_text'] = [sentiment_terms[term][1] for term in sent_terms]
        elif sentiment_terms[sent_terms[0]][0] <= -0.33:
            targets_assessments[target]['sentiment'] = 'negative'
            targets_assessments[target]['sentiment_text'] = [sentiment_terms[term][1] for term in sent_terms]
        else:
            targets_assessments[target]['sentiment'] = 'mixed'
            targets_assessments[target]['sentiment_text'] = [sentiment_terms[term][1] for term in sent_terms]

    return targets_assessments

2022-02-07 13:38:58 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2022-02-07 13:38:58 INFO: Use device: gpu
2022-02-07 13:38:58 INFO: Loading: tokenize
2022-02-07 13:38:58 INFO: Loading: pos
2022-02-07 13:38:58 INFO: Loading: lemma
2022-02-07 13:38:58 INFO: Loading: depparse
2022-02-07 13:38:59 INFO: Loading: sentiment
2022-02-07 13:38:59 INFO: Loading: constituency
2022-02-07 13:38:59 INFO: Loading: ner
2022-02-07 13:38:59 INFO: Done loading processors!


In [12]:
import pandas as pd
text = []
text.append("nice yulu ride but glitchy app")
text.append("Super expensive ..they looted me for 500 rupees for just 10 minutes ride..")
text.append("yulu had not gave my refund money which paid me though payu but money didn't receive to bank a/c if I didn't receive my yulu refund I will file an fir")
text.append("yulu had not gave my refund money which paid me though payu but money didn't receive to bank account if I didn't receive my yulu refund I will file an fir")
text.append("they are looting miney in the super saver packs. No charged vehicle are available near peak point. Bad service high prices. Complained but only excuses. Why to charge per minute in banglore traffic, better to walk or owe cycle instead of spending it on yulu. High price low service.")
text.append("great stupid developer making this app, geveloper making this app only for owner, nothing any solution for customer.")
#text.append("best battery bike ever worth off money \ud83d\udcb8 \ud83d\udcb8\ud83d\udcb8 I had used in banglore... For one day it's very useful... Am in chennai.... But yulu is not there in chennai kindly make tie up with this government it's very useful for us........")
text.append("best for daily commuters in bengaluru")
text.append("nice experience. arrangement is clear and smooth. Cycle quality is good. Only suggestion to indicate battery life once ride starts.")
text.append("great ride i reached office on time today exactly at 10am... Fabulous i loved the ride... Charges should be less... Its not distinguished how the fare is calculated...")
text.append("good for nearby places.....but your charges is high for an electric vehicle. It is like 1rs per min....pls consider")
text.append("app is crashing like anything even after updating it..")
text.append("waste of money 3rd class app very bad service")
#text.append('This is a good comment. All other comments are bad.')
#text.append('The new car is good but the mileage is bad.')
#text.append('Santa is good but banta is bad')
#text.append('Santa and banta are good and bad, respectively')
#text.append('I drove the bike.')

In [13]:
values

,target_text,assessment_text,sentiment
0,class_app,3rd,mixed
1,service,bad,negative


%time
df = pd.read_json('yulu.gplay.json')
text = list(df.content)
mod_list = []

def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def joiner(x):
    x = [x_ for x_ in x if x_ != '.']
    return ' '.join(x)

count = 0
for text_ in text:
    if text_ == 'Bhai maza aa gya paise wasul':
        text_ = 'value for money'
    text_ = deEmojify(text_)
    result = extract_aspects(text_, nlp)
    values = pd.DataFrame(result.values())
    
    if len(values):
        values['assessment_text'] = values['assessment_text'].apply(lambda x: joiner(x))
        values['mod_text'] = values['assessment_text'] + ' ' + values['target_text']
        mod_list.append(list(values['mod_text']))
    else:
        mod_list.append(text_.split('.'))

In [15]:
#df['mod_list'] = mod_list

In [16]:
#df.to_json('yulu_w_modlist.gplay.json')

In [17]:
#df[['content', 'mod_list']]

In [71]:
df = pd.read_json('yulu_w_modlist.gplay.json')[['content', 'mod_list']]

In [72]:
df = df.head(10)

In [73]:
d2vmodel = fasttext.load_model('cc.en.300.bin')
from keras.preprocessing.sequence import pad_sequences

def vectorize_comments_seq(item):
    
    word_list = item.split(' ')
    seq_vec = []
    init = True

    for word in word_list:
        if init:
            vec = d2vmodel.get_word_vector(word)
            init = False
        else:
            vec = np.vstack((vec, d2vmodel.get_word_vector(word)))
    if vec.shape == (vec.shape[0],):
        vec = vec.reshape(1, vec.shape[0])
    return pad_sequences(vec.T, maxlen=100, dtype='float').T  

In [79]:
#lol_phrase_embeddings = [[vectorize_comments_seq(m) for m in mod] for mod in df.mod_list]
k = [[]]
init = True
for mod in df.mod_list:        
    for m in mod:
        if init:
            if m != '': 
                k = vectorize_comments_seq(m)
            else:
                k = np.zeros((100, 300))
            k = k.reshape(1, 100, 300)
        else:
            if m != '': 
                k_ = vectorize_comments_seq(m)
            else:
                k_ = np.zeros((100, 300))
            k_ = k_.reshape(1, 100, 300)
            k = np.vstack((k, k_))
        init = False
        #print(m)
#print(k)

In [80]:
k.shape

(17, 100, 300)

In [76]:
df

,content,mod_list
0,App is crashing like anything even after updat...,"[crashing App, crashing anything]"
1,Nice ride,[Nice ride]
2,Super expensive ..they looted me for 500 rupee...,"[looted 500 rupee, 10 ride minute, looted ride..."
3,Expensive then petrol vehicle...,[Expensive then ... vehicle petrol]
4,For one full charge how much km v can ride,"[one full charge, much ride kilometer, ride v]"
5,Unnecessary charges I can't able to pause the ...,"[Unnecessary charge, pause ride]"
6,Amazing and comfortable experience,[Amazing experience]
7,"Very important this , next Journey to Electric...","[important , next journey vehicle, Electric jo..."
8,Great 👍,[Great ]
9,Nice feeling good,[Nice good feeling]


In [77]:
(24400, 300, 1), (1700, 300, 1), (200, 300, 1)

((24400, 300, 1), (1700, 300, 1), (200, 300, 1))

In [78]:
for mod in df.mod_list:
    print(mod, len(mod))

['crashing App', 'crashing anything'] 2
['Nice ride'] 1
['looted 500 rupee', '10 ride minute', 'looted ride minute'] 3
['Expensive then ... vehicle petrol'] 1
['one full charge', 'much ride kilometer', 'ride v'] 3
['Unnecessary charge', 'pause ride'] 2
['Amazing experience'] 1
['important , next journey vehicle', 'Electric journey vehicle'] 2
['Great '] 1
['Nice good feeling'] 1
